In [1]:
#%pip install segment-geospatial

In [2]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps

In [3]:
m = leafmap.Map(center=[29.676840, -95.369222], zoom=19)
m.add_basemap("SATELLITE")
m

Map(center=[29.67684, -95.369222], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [4]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
    print(bbox)
else:
    bbox = [-95.3704, 29.6762, -95.368, 29.6775]

[-95.3696, 29.676, -95.3689, 29.6769]


In [5]:
image = "/app/satellite.tif"

In [6]:
get_basemaps().keys()

dict_keys(['BasemapAT.basemap', 'BasemapAT.grau', 'BasemapAT.highdpi', 'BasemapAT.orthofoto', 'BasemapAT.overlay', 'BasemapAT.surface', 'BasemapAT.terrain', 'CartoDB.DarkMatter', 'CartoDB.DarkMatterNoLabels', 'CartoDB.DarkMatterOnlyLabels', 'CartoDB.Positron', 'CartoDB.PositronNoLabels', 'CartoDB.PositronOnlyLabels', 'CartoDB.Voyager', 'CartoDB.VoyagerLabelsUnder', 'CartoDB.VoyagerNoLabels', 'CartoDB.VoyagerOnlyLabels', 'CyclOSM', 'Esri.AntarcticBasemap', 'Esri.AntarcticImagery', 'Esri.ArcticImagery', 'Esri.ArcticOceanBase', 'Esri.ArcticOceanReference', 'Esri.DeLorme', 'Esri.NatGeoWorldMap', 'Esri.OceanBasemap', 'Esri.WorldGrayCanvas', 'Esri.WorldImagery', 'Esri.WorldPhysical', 'Esri.WorldShadedRelief', 'Esri.WorldStreetMap', 'Esri.WorldTerrain', 'Esri.WorldTopoMap', 'FreeMapSK', 'Gaode.Normal', 'Gaode.Satellite', 'GeoportailFrance.Adminexpress_cog_Latest', 'GeoportailFrance.Adminexpress_cog_carto_Latest', 'GeoportailFrance.Areamanagement_Zfu', 'GeoportailFrance.Areamanagement_Zus', 'G

In [7]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 01/12
Downloaded image 02/12
Downloaded image 03/12
Downloaded image 04/12
Downloaded image 05/12
Downloaded image 06/12
Downloaded image 07/12
Downloaded image 08/12
Downloaded image 09/12
Downloaded image 10/12
Downloaded image 11/12
Downloaded image 12/12
Saving GeoTIFF. Please wait...
Image saved to /app/satellite.tif


In [8]:
# image = '/path/to/your/own/image.tif'

In [10]:
m.layers[-1].visible = False  # turn off the basemap
m.add_raster(image, layer_name="Image")
m

Map(bottom=55514221.0, center=[29.676450371279678, -95.36924734711648], controls=(ZoomControl(options=['positi…

In [11]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint="sam_vit_h_4b8939.pth",
    sam_kwargs=None,
)

In [12]:
mask = "segment.tif"
sam.generate(
    image, mask, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
vector = "segment.gpkg"
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

In [ ]:
shapefile = "segment.shp"
sam.tiff_to_vector(mask, shapefile)

In [ ]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector(vector, layer_name="Vector", style=style)
m